# Kaggle CAPTCHA Solver

This notebook was created to participate in the Kaggle CAPTCHA competition. It covers data loading, preprocessing, augmentation, model building, training, and submission generation.

## 1. Импорт библиотек¶

Подключаем все необходимые библиотеки.

In [29]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/d/vasiliifede/mipt-dl-captcha/sample_submission.csv
/kaggle/input/d/vasiliifede/mipt-dl-captcha/images.npy
/kaggle/input/d/vasiliifede/mipt-dl-captcha/labels.npy
/kaggle/input/d/vasiliifede/mipt-dl-captcha/images_sub.npy


## 2. Загрузка данных
Загружаем обучающие и тестовые наборы.

In [2]:
# Загрузка датасетов
images = np.load('/kaggle/input/d/vasiliifede/mipt-dl-captcha/images.npy')
images_sub  = np.load('/kaggle/input/d/vasiliifede/mipt-dl-captcha/images_sub.npy')
labels = np.load('/kaggle/input/d/vasiliifede/mipt-dl-captcha/labels.npy')
sample_sub = pd.read_csv('/kaggle/input/d/vasiliifede/mipt-dl-captcha/sample_submission.csv')

print("images:", images.shape)
print("images_sub: ", images_sub.shape)
print('labels: ', labels.shape)
print("sample submission:", sample_sub.shape)

images: (20000, 48, 48, 3)
images_sub:  (50000, 48, 48, 3)
labels:  (20000,)
sample submission: (50000, 2)


## 3. Разбиение и нормализация
Делим данные на тренировочную и валидационную выборки и нормализуем.

In [4]:
X = images.astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(
    X, labels,
    test_size=0.15,    # 15% под валидацию
    random_state=42,   # для фиксированного разбиения
    shuffle=True,
    stratify=labels    # сбалансирует классы
)
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape,   y_val.shape)

Train: (17000, 48, 48, 3) (17000,)
Val:   (3000, 48, 48, 3) (3000,)


## 4. Создание генераторов
Аугментация данных для тренировки и простая нормализация для валидации.

In [7]:

# нормализация и аугментация для train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.7,1.3),
    channel_shift_range=0.5,    # изменяем еще цвета
    horizontal_flip=True,
    validation_split=0.15
)

# для валидации — только rescale
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15
)

train_gen = train_datagen.flow(
    images, labels,
    subset='training',
    batch_size=64,
    shuffle=True
)
val_gen = val_datagen.flow(
    images, labels,
    subset='validation',
    batch_size=64,
    shuffle=False
)

## 5. Определение модели
Используем MobileNetV2 в качестве базовой модели с дополнительными головными слоями.

In [25]:
#  модель без головы
base_model = MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape = (48, 48, 3)
)

# заморозим все слои на этапе обучения головы
base_model.trainabale = False

# добавляем головные слои
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(26, activation='softmax')(x)

# склеиваем части модели
model = models.Model(inputs=base_model.input, outputs=outputs)

/tmp/ipykernel_35/1079731079.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


In [26]:
# Компиляция модели
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', verbose=1)
]

## 6. Обучение модели
Устанавливаем обратные вызовы и обучаем модель

In [27]:
# Обучение
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.0681 - loss: 3.9497
Epoch 1: val_accuracy improved from -inf to 0.06633, saving model to best_model.h5
266/266 ━━━━━━━━━━━━━━━━━━━━ 161s 409ms/step - accuracy: 0.0682 - loss: 3.9483 - val_accuracy: 0.0663 - val_loss: 8.1903 - learning_rate: 0.0010
Epoch 2/100
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.1683 - loss: 2.9512
Epoch 2: val_accuracy improved from 0.06633 to 0.10500, saving model to best_model.h5
266/266 ━━━━━━━━━━━━━━━━━━━━ 107s 401ms/step - accuracy: 0.1684 - loss: 2.9506 - val_accuracy: 0.1050 - val_loss: 6.5922 - learning_rate: 0.0010
Epoch 3/100
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.2994 - loss: 2.4002
Epoch 3: val_accuracy did not improve from 0.10500
266/266 ━━━━━━━━━━━━━━━━━━━━ 106s 398ms/step - accuracy: 0.2993 - loss: 2.4004 - val_accuracy: 0.0933 - val_loss: 7.6516 - learning_rate: 0.0010
Epoch 4/100
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.2956 - lo

KeyboardInterrupt: 

## 7. Генерация предсказаний и создание файла для сабмита.

Модель остановлена, т.к. заметных улучшений несколько эпох не показывали, а по требованиям уменьшение learning rate уменьшался бы еще несколько раз.
Уменьшаем углеродный след <3

In [28]:
# загрузка теста и нормализация
X_test = images_sub.astype('float32') / 255.
preds = model.predict(X_test, batch_size=64)
labels_sub = preds.argmax(axis=1).astype(int)

# формирование файла
submission = sample_sub.copy()
submission['Category'] = labels_sub
submission.to_csv('Fede_VV__try_3.csv', index=False)

782/782 ━━━━━━━━━━━━━━━━━━━━ 55s 68ms/step
